In [32]:
from __future__ import print_function
from ase.io import *
import numpy as np
from ase.units import Hartree,Bohr
from ase.data import covalent_radii, chemical_symbols
from kqeq.data import uff_xi_vdw, uff_Di_vdw, uff_Xi_qeq
kcal2hartree = 0.0015936010974213599


In [33]:
mols_train = read("data/LiH_dipoles.xyz@:200",format='extxyz')
mols_valid = read("data/LiH_dipoles.xyz@200:",format='extxyz')

In [34]:
def num_grad(mol,h=0.001,direction=0,iatom=0):
    tmpmol = mol.copy()
    pos = tmpmol.get_positions()
    pos[iatom][direction] += h
    tmpmol.set_positions(pos)
    Eplus,f = calc_LJ(tmpmol)
    pos[iatom][direction] += -2.0*h
    tmpmol.set_positions(pos)
    Eminus,f = calc_LJ(tmpmol)
    pos[iatom][direction] += h
    tmpmol.set_positions(pos)
    return (Eplus-Eminus)/(2.0*h)

def num_grads(mol,h=0.001):
    f = np.zeros((len(mol),3))
    for i in range(len(mol)):
        for direction in range(3):
            f[i,direction] = num_grad(mol,h=h,direction=direction,iatom=i)
    return f

In [41]:
def calc_LJ(mol):
    xyz = mol.get_positions()/Bohr
    atoms = mol.get_atomic_numbers()
    nAt = len(atoms)

    xi_LJ = [] 
    Di_LJ = []
    for El in atoms:
        xi_LJ.append(uff_xi_vdw[chemical_symbols[El]]/Bohr)
        Di_LJ.append(uff_Di_vdw[chemical_symbols[El]]*kcal2hartree)
    xi_LJ = np.array(xi_LJ)
    Di_LJ = np.array(Di_LJ)
    
    E = 0.0
    f = np.zeros((nAt,3))
    for i in range(nAt):
        for j in range(i):
            rvec = xyz[i] - xyz[j]
            rij = np.linalg.norm(rvec)

            Dij = np.sqrt(Di_LJ[i]*Di_LJ[j])
            xij = np.sqrt(xi_LJ[i]*xi_LJ[j])
            xr6  = (xij/rij)**6
            xr12 = xr6**2
            E += Dij*(-2.*xr6+xr12)
                
            #Forces
            dVdr = 12.*(Dij/rij)*(xr6-xr12)
            f[i,:] +=  -dVdr/rij * rvec
            f[j,:] +=   dVdr/rij * rvec
            print(f)
                
    return E*Hartree,f*(Hartree/Bohr)

In [42]:
mol = mols_train[1]
E,f = calc_LJ(mol)
print(f)

[[ 0.00012898  0.00464912 -0.00475114]
 [-0.00012898 -0.00464912  0.00475114]]
[[ 0.0066325   0.2390673  -0.24431321]
 [-0.0066325  -0.2390673   0.24431321]]


In [39]:
fnum = num_grads(mol,h=0.0001)
print(fnum)

[[-0.0066325  -0.23906731  0.24431321]
 [ 0.0066325   0.23906731 -0.24431321]]


In [40]:
fnum/f

array([[-0.99999998, -1.00000003, -1.00000004],
       [-0.99999998, -1.00000003, -1.00000004]])